In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
print(type(trainset.data))
print(trainset.data.shape)
trainset.data = np.delete(trainset.data, [0], 0)
print(trainset.data.shape)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
 
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
 
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

<class 'numpy.ndarray'>
(50000, 32, 32, 3)
(49999, 32, 32, 3)


In [2]:

import torch.nn as nn
import torch.nn.functional as F
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding = 1)
        self.conv2 = nn.Conv2d(64, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding = 1)
        self.conv5 = nn.Conv2d(128, 256, 3, padding = 1)
        self.conv6 = nn.Conv2d(256, 256, 3, padding = 1)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.avgpool = nn.AvgPool2d(2, 2)
        self.globalavgpool = nn.AvgPool2d(8, 8)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.dropout50 = nn.Dropout(0.5)
        self.dropout10 = nn.Dropout(0.1)
        self.fc = nn.Linear(256, 10)
 
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn1(F.relu(self.conv2(x)))
        x = self.maxpool(x)
        x = self.dropout10(x)
        x = self.bn2(F.relu(self.conv3(x)))
        x = self.bn2(F.relu(self.conv4(x)))
        x = self.avgpool(x)
        x = self.dropout10(x)
        x = self.bn3(F.relu(self.conv5(x)))
        x = self.bn3(F.relu(self.conv6(x)))
        x = self.globalavgpool(x)
        x = self.dropout50(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
 
net = Net()

In [3]:
channel_1 = 32
channel_2 = 16
model = nn.Sequential(
    nn.Conv2d(3, channel_1, 5, padding = 2),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding = 1),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(channel_2 * 32 * 32, 10),
)

In [4]:
import torch.optim as optim
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (globalavgpool): AvgPool2d(kernel_size=8, stride=8, padding=0)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout50): Dropout(p=0.5, inplace=False)
 

In [13]:

for epoch in range(10):
 
    running_loss = 0.
    batch_size = 100
    
    for i, data in enumerate(
            torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2), 0):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        print('[%d, %5d] loss: %.4f' %(epoch + 1, (i+1)*batch_size, loss.item()))
 
print('Finished Training')

[1,   100] loss: 0.1299
[1,   200] loss: 0.1269
[1,   300] loss: 0.1096
[1,   400] loss: 0.1640
[1,   500] loss: 0.1770
[1,   600] loss: 0.1058
[1,   700] loss: 0.1352
[1,   800] loss: 0.1389
[1,   900] loss: 0.1072
[1,  1000] loss: 0.0679
[1,  1100] loss: 0.0856
[1,  1200] loss: 0.1877
[1,  1300] loss: 0.1207
[1,  1400] loss: 0.1186
[1,  1500] loss: 0.1123
[1,  1600] loss: 0.1773
[1,  1700] loss: 0.1489
[1,  1800] loss: 0.0920
[1,  1900] loss: 0.1229
[1,  2000] loss: 0.2530
[1,  2100] loss: 0.1248
[1,  2200] loss: 0.1532
[1,  2300] loss: 0.1058
[1,  2400] loss: 0.1512
[1,  2500] loss: 0.1791
[1,  2600] loss: 0.2867
[1,  2700] loss: 0.2184
[1,  2800] loss: 0.0657
[1,  2900] loss: 0.1267
[1,  3000] loss: 0.1187
[1,  3100] loss: 0.1522
[1,  3200] loss: 0.1004
[1,  3300] loss: 0.1743
[1,  3400] loss: 0.1096
[1,  3500] loss: 0.0608
[1,  3600] loss: 0.1724
[1,  3700] loss: 0.1567
[1,  3800] loss: 0.0758
[1,  3900] loss: 0.1277
[1,  4000] loss: 0.1264
[1,  4100] loss: 0.1187
[1,  4200] loss:

[1, 34500] loss: 0.1424
[1, 34600] loss: 0.0994
[1, 34700] loss: 0.1277
[1, 34800] loss: 0.1495
[1, 34900] loss: 0.1657
[1, 35000] loss: 0.1788
[1, 35100] loss: 0.2490
[1, 35200] loss: 0.1212
[1, 35300] loss: 0.0718
[1, 35400] loss: 0.1134
[1, 35500] loss: 0.1651
[1, 35600] loss: 0.1107
[1, 35700] loss: 0.0855
[1, 35800] loss: 0.1066
[1, 35900] loss: 0.1460
[1, 36000] loss: 0.1261
[1, 36100] loss: 0.0812
[1, 36200] loss: 0.1521
[1, 36300] loss: 0.1732
[1, 36400] loss: 0.1283
[1, 36500] loss: 0.1445
[1, 36600] loss: 0.1469
[1, 36700] loss: 0.1365
[1, 36800] loss: 0.0628
[1, 36900] loss: 0.2042
[1, 37000] loss: 0.1048
[1, 37100] loss: 0.1759
[1, 37200] loss: 0.0633
[1, 37300] loss: 0.1099
[1, 37400] loss: 0.1042
[1, 37500] loss: 0.0997
[1, 37600] loss: 0.2498
[1, 37700] loss: 0.1604
[1, 37800] loss: 0.1545
[1, 37900] loss: 0.1124
[1, 38000] loss: 0.0984
[1, 38100] loss: 0.0849
[1, 38200] loss: 0.1855
[1, 38300] loss: 0.1605
[1, 38400] loss: 0.1289
[1, 38500] loss: 0.1577
[1, 38600] loss:

[2, 18900] loss: 0.1339
[2, 19000] loss: 0.2019
[2, 19100] loss: 0.1175
[2, 19200] loss: 0.1166
[2, 19300] loss: 0.0869
[2, 19400] loss: 0.0584
[2, 19500] loss: 0.1519
[2, 19600] loss: 0.1931
[2, 19700] loss: 0.1534
[2, 19800] loss: 0.1682
[2, 19900] loss: 0.1368
[2, 20000] loss: 0.1376
[2, 20100] loss: 0.1385
[2, 20200] loss: 0.0980
[2, 20300] loss: 0.1798
[2, 20400] loss: 0.1543
[2, 20500] loss: 0.0715
[2, 20600] loss: 0.0687
[2, 20700] loss: 0.0845
[2, 20800] loss: 0.0623
[2, 20900] loss: 0.2525
[2, 21000] loss: 0.1150
[2, 21100] loss: 0.1000
[2, 21200] loss: 0.1418
[2, 21300] loss: 0.1613
[2, 21400] loss: 0.0820
[2, 21500] loss: 0.1223
[2, 21600] loss: 0.0972
[2, 21700] loss: 0.1813
[2, 21800] loss: 0.1408
[2, 21900] loss: 0.1902
[2, 22000] loss: 0.1284
[2, 22100] loss: 0.0989
[2, 22200] loss: 0.0835
[2, 22300] loss: 0.2115
[2, 22400] loss: 0.1129
[2, 22500] loss: 0.1737
[2, 22600] loss: 0.0996
[2, 22700] loss: 0.1240
[2, 22800] loss: 0.1636
[2, 22900] loss: 0.2268
[2, 23000] loss:

[3,  3100] loss: 0.1927
[3,  3200] loss: 0.1565
[3,  3300] loss: 0.1178
[3,  3400] loss: 0.1853
[3,  3500] loss: 0.2507
[3,  3600] loss: 0.1386
[3,  3700] loss: 0.1641
[3,  3800] loss: 0.0929
[3,  3900] loss: 0.1847
[3,  4000] loss: 0.1539
[3,  4100] loss: 0.1230
[3,  4200] loss: 0.1185
[3,  4300] loss: 0.1069
[3,  4400] loss: 0.1800
[3,  4500] loss: 0.3013
[3,  4600] loss: 0.1356
[3,  4700] loss: 0.2481
[3,  4800] loss: 0.1444
[3,  4900] loss: 0.1729
[3,  5000] loss: 0.1314
[3,  5100] loss: 0.2090
[3,  5200] loss: 0.0968
[3,  5300] loss: 0.0984
[3,  5400] loss: 0.1217
[3,  5500] loss: 0.1281
[3,  5600] loss: 0.1289
[3,  5700] loss: 0.0836
[3,  5800] loss: 0.0825
[3,  5900] loss: 0.1354
[3,  6000] loss: 0.1914
[3,  6100] loss: 0.1239
[3,  6200] loss: 0.0948
[3,  6300] loss: 0.1372
[3,  6400] loss: 0.0804
[3,  6500] loss: 0.0827
[3,  6600] loss: 0.2253
[3,  6700] loss: 0.1152
[3,  6800] loss: 0.1149
[3,  6900] loss: 0.2488
[3,  7000] loss: 0.1742
[3,  7100] loss: 0.1512
[3,  7200] loss:

[3, 37400] loss: 0.0762
[3, 37500] loss: 0.0805
[3, 37600] loss: 0.1637
[3, 37700] loss: 0.1176
[3, 37800] loss: 0.1094
[3, 37900] loss: 0.1362
[3, 38000] loss: 0.1528
[3, 38100] loss: 0.1157
[3, 38200] loss: 0.1509
[3, 38300] loss: 0.1703
[3, 38400] loss: 0.0925
[3, 38500] loss: 0.1599
[3, 38600] loss: 0.2138
[3, 38700] loss: 0.2321
[3, 38800] loss: 0.2147
[3, 38900] loss: 0.1348
[3, 39000] loss: 0.0894
[3, 39100] loss: 0.0479
[3, 39200] loss: 0.1634
[3, 39300] loss: 0.0863
[3, 39400] loss: 0.2181
[3, 39500] loss: 0.1034
[3, 39600] loss: 0.1419
[3, 39700] loss: 0.1980
[3, 39800] loss: 0.1793
[3, 39900] loss: 0.1333
[3, 40000] loss: 0.0699
[3, 40100] loss: 0.2105
[3, 40200] loss: 0.1475
[3, 40300] loss: 0.1245
[3, 40400] loss: 0.0881
[3, 40500] loss: 0.1414
[3, 40600] loss: 0.1076
[3, 40700] loss: 0.1851
[3, 40800] loss: 0.0791
[3, 40900] loss: 0.1244
[3, 41000] loss: 0.1020
[3, 41100] loss: 0.1653
[3, 41200] loss: 0.3083
[3, 41300] loss: 0.0539
[3, 41400] loss: 0.1280
[3, 41500] loss:

...

[7, 45800] loss: 0.1289
[7, 45900] loss: 0.0693
[7, 46000] loss: 0.1488
[7, 46100] loss: 0.1285
[7, 46200] loss: 0.1799
[7, 46300] loss: 0.1306
[7, 46400] loss: 0.1768
[7, 46500] loss: 0.1518
[7, 46600] loss: 0.0968
[7, 46700] loss: 0.1052
[7, 46800] loss: 0.1776
[7, 46900] loss: 0.2389
[7, 47000] loss: 0.0728
[7, 47100] loss: 0.1849
[7, 47200] loss: 0.2710
[7, 47300] loss: 0.1205
[7, 47400] loss: 0.1355
[7, 47500] loss: 0.1173
[7, 47600] loss: 0.1858
[7, 47700] loss: 0.1042
[7, 47800] loss: 0.1146
[7, 47900] loss: 0.1518
[7, 48000] loss: 0.0959
[7, 48100] loss: 0.1295
[7, 48200] loss: 0.1692
[7, 48300] loss: 0.1717
[7, 48400] loss: 0.1298
[7, 48500] loss: 0.2376
[7, 48600] loss: 0.1474
[7, 48700] loss: 0.1751
[7, 48800] loss: 0.0729
[7, 48900] loss: 0.1776
[7, 49000] loss: 0.1053
[7, 49100] loss: 0.2515
[7, 49200] loss: 0.1179
[7, 49300] loss: 0.1547
[7, 49400] loss: 0.0820
[7, 49500] loss: 0.2121
[7, 49600] loss: 0.1656
[7, 49700] loss: 0.1725
[7, 49800] loss: 0.0631
[7, 49900] loss:

[8, 30100] loss: 0.1489
[8, 30200] loss: 0.1102
[8, 30300] loss: 0.0982
[8, 30400] loss: 0.2412
[8, 30500] loss: 0.2802
[8, 30600] loss: 0.0996
[8, 30700] loss: 0.1121
[8, 30800] loss: 0.1148
[8, 30900] loss: 0.2625
[8, 31000] loss: 0.1598
[8, 31100] loss: 0.0938
[8, 31200] loss: 0.1206
[8, 31300] loss: 0.1052
[8, 31400] loss: 0.2821
[8, 31500] loss: 0.1827
[8, 31600] loss: 0.1176
[8, 31700] loss: 0.1111
[8, 31800] loss: 0.1476
[8, 31900] loss: 0.0819
[8, 32000] loss: 0.2104
[8, 32100] loss: 0.0938
[8, 32200] loss: 0.1152
[8, 32300] loss: 0.1237
[8, 32400] loss: 0.1542
[8, 32500] loss: 0.0716
[8, 32600] loss: 0.1140
[8, 32700] loss: 0.1619
[8, 32800] loss: 0.0902
[8, 32900] loss: 0.0713
[8, 33000] loss: 0.1316
[8, 33100] loss: 0.1527
[8, 33200] loss: 0.1146
[8, 33300] loss: 0.1403
[8, 33400] loss: 0.0505
[8, 33500] loss: 0.2915
[8, 33600] loss: 0.1926
[8, 33700] loss: 0.1116
[8, 33800] loss: 0.1387
[8, 33900] loss: 0.1465
[8, 34000] loss: 0.1317
[8, 34100] loss: 0.2046
[8, 34200] loss:

[9, 14400] loss: 0.1331
[9, 14500] loss: 0.2254
[9, 14600] loss: 0.1342
[9, 14700] loss: 0.1686
[9, 14800] loss: 0.1108
[9, 14900] loss: 0.0765
[9, 15000] loss: 0.1967
[9, 15100] loss: 0.1345
[9, 15200] loss: 0.2200
[9, 15300] loss: 0.1567
[9, 15400] loss: 0.1292
[9, 15500] loss: 0.1243
[9, 15600] loss: 0.2319
[9, 15700] loss: 0.1275
[9, 15800] loss: 0.1072
[9, 15900] loss: 0.1069
[9, 16000] loss: 0.2045
[9, 16100] loss: 0.0854
[9, 16200] loss: 0.2848
[9, 16300] loss: 0.1136
[9, 16400] loss: 0.1373
[9, 16500] loss: 0.1327
[9, 16600] loss: 0.2049
[9, 16700] loss: 0.1543
[9, 16800] loss: 0.0717
[9, 16900] loss: 0.1585
[9, 17000] loss: 0.1353
[9, 17100] loss: 0.1525
[9, 17200] loss: 0.0797
[9, 17300] loss: 0.1238
[9, 17400] loss: 0.2024
[9, 17500] loss: 0.1412
[9, 17600] loss: 0.1717
[9, 17700] loss: 0.0749
[9, 17800] loss: 0.0739
[9, 17900] loss: 0.1827
[9, 18000] loss: 0.1177
[9, 18100] loss: 0.0982
[9, 18200] loss: 0.1697
[9, 18300] loss: 0.1439
[9, 18400] loss: 0.1186
[9, 18500] loss:

[9, 49200] loss: 0.0814
[9, 49300] loss: 0.1326
[9, 49400] loss: 0.1449
[9, 49500] loss: 0.1676
[9, 49600] loss: 0.1232
[9, 49700] loss: 0.1020
[9, 49800] loss: 0.1468
[9, 49900] loss: 0.0806
[9, 50000] loss: 0.1812
[10,   100] loss: 0.1212
[10,   200] loss: 0.1246
[10,   300] loss: 0.1785
[10,   400] loss: 0.1376
[10,   500] loss: 0.1806
[10,   600] loss: 0.1601
[10,   700] loss: 0.0651
[10,   800] loss: 0.1069
[10,   900] loss: 0.1821
[10,  1000] loss: 0.1294
[10,  1100] loss: 0.1061
[10,  1200] loss: 0.1611
[10,  1300] loss: 0.1452
[10,  1400] loss: 0.1576
[10,  1500] loss: 0.1287
[10,  1600] loss: 0.1163
[10,  1700] loss: 0.0970
[10,  1800] loss: 0.1143
[10,  1900] loss: 0.1589
[10,  2000] loss: 0.1416
[10,  2100] loss: 0.1412
[10,  2200] loss: 0.0850
[10,  2300] loss: 0.0992
[10,  2400] loss: 0.1555
[10,  2500] loss: 0.0898
[10,  2600] loss: 0.1089
[10,  2700] loss: 0.1671
[10,  2800] loss: 0.1418
[10,  2900] loss: 0.0998
[10,  3000] loss: 0.0984
[10,  3100] loss: 0.1023
[10,  320

[10, 32600] loss: 0.1285
[10, 32700] loss: 0.1059
[10, 32800] loss: 0.1686
[10, 32900] loss: 0.1286
[10, 33000] loss: 0.1139
[10, 33100] loss: 0.0662
[10, 33200] loss: 0.0997
[10, 33300] loss: 0.1551
[10, 33400] loss: 0.0608
[10, 33500] loss: 0.0755
[10, 33600] loss: 0.1110
[10, 33700] loss: 0.0475
[10, 33800] loss: 0.1814
[10, 33900] loss: 0.1762
[10, 34000] loss: 0.1052
[10, 34100] loss: 0.1048
[10, 34200] loss: 0.1771
[10, 34300] loss: 0.1314
[10, 34400] loss: 0.1225
[10, 34500] loss: 0.1658
[10, 34600] loss: 0.2046
[10, 34700] loss: 0.1135
[10, 34800] loss: 0.1548
[10, 34900] loss: 0.1232
[10, 35000] loss: 0.3806
[10, 35100] loss: 0.1598
[10, 35200] loss: 0.2491
[10, 35300] loss: 0.1475
[10, 35400] loss: 0.1649
[10, 35500] loss: 0.1543
[10, 35600] loss: 0.1354
[10, 35700] loss: 0.1172
[10, 35800] loss: 0.1428
[10, 35900] loss: 0.2055
[10, 36000] loss: 0.1596
[10, 36100] loss: 0.1326
[10, 36200] loss: 0.1137
[10, 36300] loss: 0.1242
[10, 36400] loss: 0.1388
[10, 36500] loss: 0.0739


In [15]:
torch.save(net, 'cifar10.pkl')
net = torch.load('cifar10.pkl')

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
 
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
 
 
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the 10000 test images: 69 %
Accuracy of plane : 72 %
Accuracy of   car : 86 %
Accuracy of  bird : 57 %
Accuracy of   cat : 59 %
Accuracy of  deer : 63 %
Accuracy of   dog : 61 %
Accuracy of  frog : 71 %
Accuracy of horse : 72 %
Accuracy of  ship : 68 %
Accuracy of truck : 78 %
